In [15]:
import csv
import glob
import shutil
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

# Create a folder with format (11-November 2018) in the directory 
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')
dir_name = 'S://TANF Reports//!2020 TANF Reports//'
os.chdir(dir_name)


folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


# Move all .csv files in Downloads folder to S:drive(destination) folder
path = 'C://Users//nnagaraj//Downloads'
os.chdir(path)
source = os.listdir(path)
destination = os.path.join('S://TANF Reports//!2020 TANF Reports', folder_name )

for files in source:  
    if files.endswith(".csv"):
        shutil.move(files, destination)  
        

# Convert all .csv files in S:drive to Excel        
os.chdir(destination)        
for filename in os.listdir(destination):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(destination, filename))
        #df['Job Ret Date'] = pd.to_datetime(df['Job Ret Date']) # for JRM Mailing & Universe
        #df['Begin Date'] = pd.to_datetime(df['Begin Date']) # for JRM Universe
        #df = df.sort_values(by = ['local office', 'Name'])
        #df = df.sort_values(by = ['local office', 'Last Name']) # for TEA Adults with Assigned Activity
        x = filename.replace("-", " ") + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title()
        writer = pd.ExcelWriter(str(x).title().replace(".Csv", "")+".xlsx")
        df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False, header = filename)
        writer.save()
        continue
    else:
        continue       

# Now that all csv have been converted to excel, delete all the csv files in the directory        
files = [file for file in glob.glob("*.csv") if not file.endswith("0.csv")]
for file in files:
    os.remove(file)

# Closure Due to Employment by County

### Merge Closure due to employment and Closure due to employment with address 

In [8]:
import csv
import pandas as pd
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')

closure_emp = pd.read_excel('Closures Due To Employment - ' + prev.strftime('%B') + ' ' + now.strftime('%Y') + '.xlsx')
closure_emp_add = pd.read_excel('Closures Due To Employment With Address - ' + prev.strftime('%B') + ' ' + now.strftime('%Y') + '.xlsx')

df = pd.merge(closure_emp, closure_emp_add, how = 'inner', on = 'County')

df = df.drop(['SSN', 'Address 1', 'Address 2', 'Address 3', 'Zip'], axis = 1)


# At first use duplicated to check for duplicated values in your dataframe. Then select which columns you want to be changed
is_duplicated = df.apply(lambda x: x.duplicated())
df_map = df.where(~is_duplicated, " ")
df[['County', '# Closures Due to Employment', 'Total Closures', 'Percentage']] = df_map.drop(['Case No', 'Casehead Name'], axis = 1)
df.head(4)

# Write created df to excel
writer = pd.ExcelWriter('Closures Due to Employment by County' + ' - ' + prev.strftime('%B') + ' ' + now.strftime('%Y') + '.xlsx')
df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False)#, header = filename 
writer.save()


# Delete file Closures Due To Employment - December 2018.xlsx
os.remove('Closures Due To Employment - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')

# Extracting Deferred, Exempt and Sanction from Disaggrigated file in S:drive

In [5]:
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')

folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
destination = os.path.join('S://TANF Reports//!2020 TANF Reports', folder_name )


os.chdir(destination)
os.getcwd()


def riter(x):
    x.to_excel(writer, sheet_name = prev.strftime('%B'), index = False)
    writer.save()
    
df = pd.read_excel('Dissaggregated Def_Xmpt_Sanc By Local Office'+ ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')

    
sanction = df[df['Deferred / Exempt / Sanction'] == 'Sanction']
sanction = sanction.sort_values(by = ['Local Office'])
writer = pd.ExcelWriter('TEA Adults Sanction by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')    
riter(sanction)

exempt = df[df['Deferred / Exempt / Sanction'] == 'Exempt']
exempt = exempt.sort_values(by = ['Local Office'])
writer = pd.ExcelWriter('TEA Adults Exempt by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')
riter(exempt)

deferred = df[df['Deferred / Exempt / Sanction'] == 'Deferred']
deferred = deferred.sort_values(by = ['Local Office'])
writer = pd.ExcelWriter('TEA Adults Deferred by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')
riter(deferred)

# Delete Disaggregated Def_Xmpt_Sanc By Local Office - June 2019.xlsx
if os.path.exists('Dissaggregated Def_Xmpt_Sanc By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx'):
    os.remove('Dissaggregated Def_Xmpt_Sanc By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')
else:
    print("The file does not exist")



## Download *<u>z_Caseload Report (SQL)<u>* & *<u>z_Sanctioned Adults by Local Office (SQL)<u>* for the following

In [30]:

import csv
import glob
import shutil
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

# Create a folder with format (11-November 2018) in the directory 
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')
dir_name = 'S://TANF Reports//!2020 TANF Reports//'
os.chdir(dir_name)


folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


# Move all .csv files in Downloads folder to S:drive(destination) folder
path = 'C://Users//nnagaraj//Downloads'
os.chdir(path)
source = os.listdir(path)
destination = os.path.join('S://TANF Reports//!2020 TANF Reports', folder_name )


for files in source:  
    if files.endswith(".csv"):
        shutil.move(files, destination)  


# Convert all .csv files in S:drive to Excel        
os.chdir(destination)        
for filename in os.listdir(destination):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(destination, filename))

        x = filename.replace("-", " ") + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title()
        writer = pd.ExcelWriter(str(x).title().replace(".Csv", "")+".xlsx")
        df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False, header = filename)
        writer.save()
        continue
    else:
        continue       

# Now that all csv have been converted to excel, delete all the csv files in the directory        
files = [file for file in glob.glob("*.csv") if not file.endswith("0.csv")]
for file in files:
    os.remove(file)
    
    
# Load Excel
sanc_count = pd.read_excel('Z_Sanctioned Adults By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')
caseload_count = pd.read_excel('Z_Time Limited Caseload Report_1' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx') 

# sanction file - Drop columns 
sanction_count = sanc_count.drop(['Casehead Name','SSN', 'Name','State Count', 'Federal Count'], axis='columns')

# Dataframe to be used in excel tab
df_sanction_count = sanction_count.groupby('Local Office').agg({'Case Number': pd.Series.nunique}).reset_index()
#df_sanction_count = sanction_count.rename(index=str, columns={'Case Number': 'Sanction Count'})

# caseload - Remove Child Only Cases
#caseload_count = caseload_count[caseload_count.Type != 'Child Only']

# caseload - Rename 'name' to 'Case Manager' & 'local office' to 'Local Office'
caseload_count = caseload_count.rename(columns = {'name': 'Case Manager', 'local office': 'Local Office'})

# caseload - Create Program column based on +ve or -ve case number
caseload_count['Program'] = caseload_count['Case Number'].apply(lambda x: 'TEA' if x >= 0 else 'Work Pays')

# Dataframe to be used in excel tab
df_case_count = caseload_count.groupby(['Local Office', 'Case Manager','Program Type']).agg({'Case Number': pd.Series.nunique}).reset_index()
#df_case_count = caseload_count.rename(index = str,  columns={'Case Number': 'Caseload Count'})

# caseload aggrigate
cl_agg = caseload_count.groupby('Local Office').agg({'Case Manager': pd.Series.nunique,'Case Number': pd.Series.nunique }).reset_index()
cl_agg = cl_agg.rename(index = str, columns={'Case Manager': 'Number of WFS', 'Case Number': 'Caseload Count'})

# Merge sanction and caseload files
agg_frame = cl_agg.merge(df_sanction_count, on='Local Office')

# Calculations
agg_frame['Average Cases per WFS'] = agg_frame['Caseload Count'].div(agg_frame['Number of WFS']).round(1)
agg_frame['Aggregate Percentage Sanction'] = (100*(agg_frame['Case Number'])).div(agg_frame['Caseload Count']).round(1)

# Reorder columns
agg_frame = agg_frame[['Local Office','Case Number', 'Caseload Count', 'Number of WFS', 'Average Cases per WFS', 'Aggregate Percentage Sanction']]

# Stats
# Creating variables for dataframe
max_caseload = agg_frame.loc[(agg_frame['Caseload Count'].idxmax())].to_frame().T.reset_index()
min_caseload = agg_frame.loc[(agg_frame['Caseload Count'].idxmin())].to_frame().T.reset_index()
max_caseload_wfs = agg_frame.loc[(agg_frame['Average Cases per WFS'].idxmax())].to_frame().T.reset_index()
min_caseload_wfs = agg_frame.loc[(agg_frame['Average Cases per WFS'].idxmin())].to_frame().T.reset_index()
max_per_san = agg_frame.loc[(agg_frame['Aggregate Percentage Sanction'].idxmax())].to_frame().T.reset_index()
min_per_san = agg_frame.loc[(agg_frame['Aggregate Percentage Sanction'].idxmin())].to_frame().T.reset_index()

# Local Office Column
lo_max_caseload = max_caseload.loc[0,'Local Office']
lo_min_caseload = min_caseload.loc[0,'Local Office']
lo_max_caseload_wfs = max_caseload_wfs.loc[0,'Local Office']
lo_min_caseload_wfs = min_caseload_wfs.loc[0,'Local Office']
lo_max_per_san = max_per_san.loc[0,'Local Office']
lo_min_per_san = min_per_san.loc[0,'Local Office']

# Number column
num_max_caseload = max_caseload.loc[0,'Caseload Count']
num_min_caseload = min_caseload.loc[0,'Caseload Count']
num_max_caseload_wfs = max_caseload_wfs.loc[0,'Average Cases per WFS']
num_min_caseload_wfs = min_caseload_wfs.loc[0,'Average Cases per WFS']
num_max_per_san = max_per_san.loc[0,'Aggregate Percentage Sanction']
num_min_per_san = min_per_san.loc[0,'Aggregate Percentage Sanction']

# DataFrame
stat_frame = pd.DataFrame({'Summary_Matrix': ['LO with Maximum Caseload',
                                              'LO with Minimum Caseload',
                                             'LO with Maximum Caseload, WFS Ratio',
                                             'LO with Minimum Caseload, WFS Ratio',
                                             'LO with Maximum Sanction Percentage',
                                             'LO with Minimum Sanction Percentage'],
                           'Local_Office':[lo_max_caseload,
                                           lo_min_caseload,
                                           lo_max_caseload_wfs,
                                           lo_min_caseload_wfs,
                                           lo_max_per_san,
                                           lo_min_per_san],
                           'Number': [num_max_caseload,
                                      num_min_caseload,
                                      num_max_caseload_wfs,
                                      num_min_caseload_wfs,
                                      num_max_per_san,
                                      num_min_per_san]},
                         columns = ['Summary_Matrix','Local_Office','Number'])

# Save statistics in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx 
file_name = 'Caseload Breakdown by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='openpyxl')

# Save dataframe in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx
if os.path.exists(file_name):
    book = openpyxl.load_workbook(file_name)
    writer.book = book
    
# Save dataframe in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx    
df_sanction_count.to_excel(writer, sheet_name= str_prev_month + '_sanction', index=False)
df_case_count.to_excel(writer, sheet_name= str_prev_month + '_caseload', index=False)
agg_frame.to_excel(writer, sheet_name= str_prev_month + '_aggrigate', index=False)
stat_frame.to_excel(writer, sheet_name= str_prev_month + '_stats', index=False)
writer.save()
writer.close()

In [43]:
df_case_count

,Local Office,Case Manager,Program,Case Number
0,Arkadelphia,Tonya Campbell,TEA,3
1,Arkadelphia,Tonya Campbell,Work Pays,1
2,Batesville,Judy Fulbright,TEA,8
3,Batesville,Judy Fulbright,Work Pays,3
4,Batesville,Kim Childress,TEA,7
5,Batesville,Kim Childress,Work Pays,2
6,Batesville,Rosa Montgomery,TEA,7
7,Batesville,Teresa Staggs,TEA,8
8,Batesville,Teresa Staggs,Work Pays,2
9,Benton,Angelo Zambrano,TEA,7


In [41]:
df_case_count

,Local Office,Case Manager,Program,Case Number
0,Arkadelphia,Tonya Campbell,TEA,3
1,Arkadelphia,Tonya Campbell,Work Pays,1
2,Batesville,Judy Fulbright,TEA,8
3,Batesville,Judy Fulbright,Work Pays,3
4,Batesville,Kim Childress,TEA,7
5,Batesville,Kim Childress,Work Pays,2
6,Batesville,Rosa Montgomery,TEA,7
7,Batesville,Teresa Staggs,TEA,8
8,Batesville,Teresa Staggs,Work Pays,2
9,Benton,Angelo Zambrano,TEA,7


In [9]:
df_sanction_count = sanction_count.groupby('Local Office').agg({'Case Number': pd.Series.nunique}).reset_index()
#df_sanction_count = sanction_count.rename(index=str, columns={'Case Number': 'Sanction Count'})
df_sanction_count

,Local Office,Case Number
0,Arkadelphia,1
1,Batesville,8
2,Blytheville,5
3,Conway,6
4,El Dorado,3
5,Fayetteville,12
6,Forrest City,10
7,Fort Smith,23
8,Harrison,7
9,Helena,14


In [16]:
"""
import csv
import glob
import shutil
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

# Create a folder with format (11-November 2018) in the directory 
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')
dir_name = 'S://TANF Reports//!2019 TANF Reports//'
os.chdir(dir_name)


folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


# Move all .csv files in Downloads folder to S:drive(destination) folder
path = 'C://Users//nnagaraj//Downloads'
os.chdir(path)
source = os.listdir(path)
destination = os.path.join('S://TANF Reports//!2019 TANF Reports', folder_name )


for files in source:  
    if files.endswith(".csv"):
        shutil.move(files, destination)  


# Convert all .csv files in S:drive to Excel        
os.chdir(destination)        
for filename in os.listdir(destination):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(destination, filename))

        x = filename.replace("-", " ") + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title()
        writer = pd.ExcelWriter(str(x).title().replace(".Csv", "")+".xlsx")
        df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False, header = filename)
        writer.save()
        continue
    else:
        continue       

# Now that all csv have been converted to excel, delete all the csv files in the directory        
files = [file for file in glob.glob("*.csv") if not file.endswith("0.csv")]
for file in files:
    os.remove(file)
    
    
# Load Excel
sanc_count = pd.read_excel('sanctioned Cases By Local Office and Manager' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')
caseload_count = pd.read_excel('Caseload Breakdown By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')

# Sanction count in each local office
san_grp = sanc_count.groupby('local_office').agg({'sanction_count':'sum'}).reset_index()

# Total caseload (TEA/WorkPays) in each local office
caseload_grp = caseload_count.groupby('local_office').agg({'caseload_count':'sum'}).reset_index()

# Create a new dataframe df with columns: caseload_grp and san_grp
df = pd.merge(san_grp, caseload_grp, how = 'inner', on = 'local_office')

# Total wfs in each local office
total_wfs = caseload_count.groupby('local_office').agg({'name':'count'}).reset_index()

# Merge all columns to dataframe df2
df2 = pd.merge(df, total_wfs, how = 'inner', on ='local_office')

# Insert calculated columns to df2
df2['avg_case'] = df2['caseload_count'].div(df2['name'])

df2['per_san'] = (100*(df2['sanction_count'])).div(df2['caseload_count'])


# Save statistics in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx 
file_name = 'Caseload Breakdown by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='openpyxl')


if os.path.exists(file_name):
    book = openpyxl.load_workbook(file_name)
    writer.book = book
    
df2.to_excel(writer, sheet_name= str_prev_month + '_statistics', index=False)
writer.save()
writer.close()


# Creating variables for dataframe
max_caseload = df2.loc[(df2['caseload_count'].idxmax())].to_frame().T.reset_index()
min_caseload = df2.loc[(df2['caseload_count'].idxmin())].to_frame().T.reset_index()
max_caseload_wfs = df2.loc[(df2['avg_case'].idxmax())].to_frame().T.reset_index()
min_caseload_wfs = df2.loc[(df2['avg_case'].idxmin())].to_frame().T.reset_index()
max_per_san = df2.loc[(df2['per_san'].idxmax())].to_frame().T.reset_index()
min_per_san = df2.loc[(df2['per_san'].idxmin())].to_frame().T.reset_index()

"""
print('LO with maximum caseload count :\n{}'.format(max_caseload))
print('\n\nLO with minimum caseload count :\n{}'.format(min_caseload))
print('\n\nLO with maximum caseload, WFS Ratio :\n{}'.format(max_caseload_wfs)) 
print('\n\nLO with minimum caseload, WFS Ratio :\n{}'.format(min_caseload_wfs)) 
print('\n\nLO with maximum Sanction Percentage :\n{}'.format(max_per_san))
print('\n\nLO with minimum Sanction Percentage :\n{}'.format(min_per_san))
"""

# Local Office Column
lo_max_caseload = max_caseload.loc[0,'local_office']
lo_min_caseload = min_caseload.loc[0,'local_office']
lo_max_caseload_wfs = max_caseload_wfs.loc[0,'local_office']
lo_min_caseload_wfs = min_caseload_wfs.loc[0,'local_office']
lo_max_per_san = max_per_san.loc[0,'local_office']
lo_min_per_san = min_per_san.loc[0,'local_office']

# Number column
num_max_caseload = max_caseload.loc[0,'caseload_count']
num_min_caseload = min_caseload.loc[0,'caseload_count']
num_max_caseload_wfs = max_caseload_wfs.loc[0,'avg_case']
num_min_caseload_wfs = min_caseload_wfs.loc[0,'avg_case']
num_max_per_san = max_per_san.loc[0,'per_san']
num_min_per_san = min_per_san.loc[0,'per_san']

# DataFrame
stat_frame = pd.DataFrame({'Summary_Matrix': ['LO with Maximum Caseload',
                                              'LO with Minimum Caseload',
                                             'LO with Maximum Caseload, WFS Ratio',
                                             'LO with Minimum Caseload, WFS Ratio',
                                             'LO with Maximum Sanction Percentage',
                                             'LO with Minimum Sanction Percentage'],
                           'Local_Office':[lo_max_caseload,
                                           lo_min_caseload,
                                           lo_max_caseload_wfs,
                                           lo_min_caseload_wfs,
                                           lo_max_per_san,
                                           lo_min_per_san],
                           'Number': [num_max_caseload,
                                      num_min_caseload,
                                      num_max_caseload_wfs,
                                      num_min_caseload_wfs,
                                      num_max_per_san,
                                      num_min_per_san]},
                         columns = ['Summary_Matrix','Local_Office','Number'])
                             
            
# Save dataframe in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx
file_name = 'Caseload Breakdown by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='openpyxl')

if os.path.exists(file_name):
    book = openpyxl.load_workbook(file_name)
    writer.book = book
    
stat_frame.to_excel(writer, sheet_name = str_prev_month + '_summary', index=False)
writer.save()
writer.close()
"""